# Cadence Effects

In [ ]:
import sys
sys.path.insert(0, '../')

import numpy as np
import sncosmo
from astropy.table import Table

from sn_analysis import filters
from sn_analysis import modeling
from sn_analysis import plasticc_data
from sn_analysis import sn_magnitudes

filters.register_lsst_filters(force=True)


## Atmospheric Variability


In [ ]:
# Todo

## Simulating Light-Curves

Instead of evaluating different cadences from scratch, we rely on data from the PLaSTICC simulations.

In [ ]:
plasticc_data.get_available_cadences()


The full simulation datasets include simulations for multiple models. Since we don't need this extra information, we only use data for simulated normal SNe (Model number 11).

In [ ]:
plasticc_lc = next(plasticc_data.iter_lc_for_cadence_model('alt_sched_rolling', model=11, verbose=False))


In [ ]:
plasticc_lc.meta


In [ ]:
plasticc_lc


As as first step, we reformat the data to be compatible with `sncosmo`.

In [ ]:
def iter_formatted_lcs(cadence, verbose=False):
    """Iterate over """
    for pointing in plasticc_data.iter_lc_for_cadence_model(cadence, verbose=verbose):
        lc = Table({
            'time': pointing['MJD'],
            'band': ['lsst_hardware_' + f.lower().strip() for f in pointing['FLT']],
            'flux': pointing['FLUXCAL'],
            'fluxerr': pointing['FLUXCALERR'],
            'zp': pointing['ZEROPT'],
            'photflag': pointing['PHOTFLAG']
        })
        
        lc['zpsys'] = 'AB'
        lc.meta = pointing.meta
        yield lc
        

In [ ]:
formatted_lc = next(iter_formatted_lcs('alt_sched_rolling'))


In [ ]:
sncosmo.plot_lc(formatted_lc);


We note that many of the above data points are flagged as being non-detections (`photflag == 0`). Here are just the detections.

In [ ]:
sncosmo.plot_lc(formatted_lc[formatted_lc['photflag'] != 0]);


Since we need to add in our own atmospheric variability the pre-tabulated flux values above are of limited use. However, we can use the PLaSTICC light-curves to establish the cadence and model parameters for each simulated light-curve. This information can then be used to simulate our own light-curves with `sncosmo`.

Most of the data we need can be pulled directly from the light-curve or it's meta data. The one exception is the gain and skynoise, which need to be calculated:

$$ \sigma_f = \sqrt{\sigma_s ^ 2 + \frac{|f|}{gain}} $$
$$ \sigma_f^2 = \sigma_s^2 + \frac{|f|}{gain} $$
$$ \sigma_f^2 - \frac{|f|}{gain} = \sigma_s^2 $$
$$ \sqrt{\sigma_f^2 - \frac{|f|}{gain}} = \sigma_s $$


In [ ]:
def iter_sncosmo_lc(cadence, verbose=True):
    model = sncosmo.Model('Salt2')
    for formated_lc in iter_formatted_lcs(cadence, verbose=verbose):
        observations = formatted_lc[['band', 'time', 'zp', 'zpsys']]
        observations['gain'] = 10
        observations['skynoise'] = np.sqrt(formatted_lc['fluxerr'] ** 2 - np.abs(formatted_lc['flux']) / observations['gain'])

        params = [{
            't0': formatted_lc.meta['PEAKMJD'], 
            'x0': formatted_lc.meta['SIM_SALT2x0'], 
            'x1': formatted_lc.meta['SIM_SALT2x1'],
            'c': formatted_lc.meta['SIM_SALT2c'], 
            'z': formatted_lc.meta['REDSHIFT_FINAL']
        }]

        yield sncosmo.realize_lcs(observations, model, params)[0]


In [ ]:
sncosmo_lc = next(iter_sncosmo_lc('alt_sched_rolling', verbose=False))


In [ ]:
sncosmo.plot_lc(sncosmo_lc);


In [ ]:
sncosmo_lc


## Light-Curve Fits

In [ ]:
bands = ['lsst_hardware_' + b for b in 'ugrizy']

# Iterator over simulated light-curves
light_curves = iter_sncosmo_lc('alt_sched_rolling')

# Fit light curves
vparams = ['x0', 'x1', 'c']
fitted_mag, fitted_params = sn_magnitudes.fit_mag(
        'salt2', light_curves, vparams, bands=bands)
